# Option A - Generate Spheres from a CSV

In [12]:
out_dir = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/TMS-DBS_studies_alzheimers/derivatives/roi'
mask_path = '/Users/cu135/hires_backdrops/mni/MNI152_T1_2mm_brain_mask.nii'

In [2]:
# Specify the path to your CSV file containing NIFTI paths
input_csv_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/TMS-DBS_studies_alzheimers/metadata/master_list.xlsx'
sheet = 'Sheet1'

In [3]:
from calvin_utils.permutation_analysis_utils.statsmodels_palm import CalvinStatsmodelsPalm
# Instantiate the PalmPrepararation class
cal_palm = CalvinStatsmodelsPalm(input_csv_path=input_csv_path, output_dir=out_dir, sheet=sheet)
# Call the process_nifti_paths method
data_df = cal_palm.read_and_display_data()
data_df

,Study,DOI,RCT_Design,Age_range,CDR,Active_N,Sham_N,Mean_Age,Active_Age,Shame_Age,...,Total_pulses,Overall_Cognition_Test,Overall_Cognition_Cohen_s_D,Verbal_Memory_Test,Verbal_Memory_Cohen_s_D,Stimulation_site,Stim_Coordinate,Included,Exclusion_Reason,roi_path
0,"Koch et al., 2022",10.1093/brain/awac285,Parallel,50–85,0.5–1,25.0,25.0,75.0,75 ± 5.6,72.3 ± 7.2,...,51200.0,ADAS,0.359000,NaN,NaN,Precuneus,"[0, -70, 44]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...
1,"Liu et al., 2022",10.1093/cercor/bhab371,Parallel,50–85,0.5–1,25.0,12.0,67.0,67.28 ± 7.74,72.08 ± 7.3,...,28800.0,ADAS,0.839000,NaN,NaN,Bilateral dorsal AG,"[-61, -51, 39], [61, -51, 39]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...
2,"Li et al., 2021",10.1016/j.brs.2021.01.012,Parallel,NaN,NaN,37.0,38.0,65.0,65.97 ± 8.47,64.58 ± 7.88,...,60000.0,ADAS,0.546000,NaN,NaN,Left DLPFC,"[-44,40,29]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...
3,"Wu et al., 2022",10.1016/j.brs.2021.11.007,Parallel,≧50,≦2,24.0,23.0,66.0,66.46 ± 8.25,66.35 ± 7.99,...,25200.0,MMSE,0.608000,RAVLT,0.876,Left DLPFC,"[-38, 44, 26]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...
4,"Zhao et al., 2017",10.18632/oncotarget.13060,Parallel,NaN,1–2,17.0,13.0,69.0,69.3 ± 5.8,71.4 ± 5.2,...,12000.0,ADAS,0.426000,NaN,NaN,Parietal lobe (P3/P4) and posterior temporal l...,"[-39.5, -76.3, 47.4], [59.3, -67.3, 3.8]",0.0,Multifocal,/Users/cu135/Partners HealthCare Dropbox/Calvi...
5,"Vecchio et al, 2022",10.1007/s11357-021-00508-w,Parallel,NaN,0.5–2,30.0,16.0,71.0,71.07 ± 1.25,75.2 ± 2.29,...,45000.0,ADAS,0.294000,NaN,NaN,"Broca, Wernicke, Bilateral DLPFC, Bilateral pSAC",NaN,0.0,Multifocal,NaN
6,"Rabey et al., 2013",10.1007/s00702-012-0902-z,Parallel,NaN,1,7.0,8.0,72.0,72.6 ± 8.9,75.4 ± 9.07,...,70200.0,MMSE,1.200000,NaN,NaN,"Broca, Wernicke, Bilateral DLPFC, Bilateral pSAC",NaN,0.0,Multifocal,NaN
7,"Zhang et.al., 2019",10.1007/s00702-019-02022-y,Parallel,NaN,NaN,15.0,13.0,69.0,69 ± 8.19,68.54 ± 7.93,...,20000.0,MMSE,1.200000,NaN,NaN,"Left DLPFC, left lateral temporal lobe",NaN,0.0,Multifocal,NaN
8,"Wei et al., 2022",10.1016/j.psychres.2022.114721,Parallel,55–85,0.5–2,29.0,27.0,70.0,70 ± 8.68,71.67 ± 7.16,...,8000.0,ADAS,0.100000,PVLT,-0.022,Left lateral parietal location,"[-47, -68, 36]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...
9,"Yao et al., 2022",10.1016/j.brs.2022.06.004,Parallel,60–80,Active:1.07 ± 0.53 Sham: 1.20 ± 0.59,15.0,12.0,63.0,63.87 ± 6.85,67.6 ± 7.88,...,40000.0,ADAS,0.702915,RAVLT,0.590,Bilateral cerebellum,"[30, -88, -35], [-30, -88, -35]",NaN,NaN,NaN


In [17]:
data_df

,Study,DOI,RCT_Design,Age_range,CDR,Active_N,Sham_N,Mean_Age,Active_Age,Shame_Age,...,Overall_Cognition_Test,Overall_Cognition_Cohen_s_D,Verbal_Memory_Test,Verbal_Memory_Cohen_s_D,Stimulation_site,Stim_Coordinate,Included,Exclusion_Reason,roi_path,Subject
0,"Koch et al., 2022",10.1093/brain/awac285,Parallel,50–85,0.5–1,25.0,25.0,75.0,75 ± 5.6,72.3 ± 7.2,...,ADAS,0.359000,NaN,NaN,Precuneus,"[0, -70, 44]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...,Koch_et_al___2022
1,"Liu et al., 2022",10.1093/cercor/bhab371,Parallel,50–85,0.5–1,25.0,12.0,67.0,67.28 ± 7.74,72.08 ± 7.3,...,ADAS,0.839000,NaN,NaN,Bilateral dorsal AG,"[-61, -51, 39], [61, -51, 39]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...,Liu_et_al___2022
2,"Li et al., 2021",10.1016/j.brs.2021.01.012,Parallel,NaN,NaN,37.0,38.0,65.0,65.97 ± 8.47,64.58 ± 7.88,...,ADAS,0.546000,NaN,NaN,Left DLPFC,"[-44,40,29]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...,Li_et_al___2021
3,"Wu et al., 2022",10.1016/j.brs.2021.11.007,Parallel,≧50,≦2,24.0,23.0,66.0,66.46 ± 8.25,66.35 ± 7.99,...,MMSE,0.608000,RAVLT,0.876,Left DLPFC,"[-38, 44, 26]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...,Wu_et_al___2022
4,"Zhao et al., 2017",10.18632/oncotarget.13060,Parallel,NaN,1–2,17.0,13.0,69.0,69.3 ± 5.8,71.4 ± 5.2,...,ADAS,0.426000,NaN,NaN,Parietal lobe (P3/P4) and posterior temporal l...,"[-39.5, -76.3, 47.4], [59.3, -67.3, 3.8]",0.0,Multifocal,/Users/cu135/Partners HealthCare Dropbox/Calvi...,Zhao_et_al___2017
8,"Wei et al., 2022",10.1016/j.psychres.2022.114721,Parallel,55–85,0.5–2,29.0,27.0,70.0,70 ± 8.68,71.67 ± 7.16,...,ADAS,0.100000,PVLT,-0.022,Left lateral parietal location,"[-47, -68, 36]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...,Wei_et_al___2022
9,"Yao et al., 2022",10.1016/j.brs.2022.06.004,Parallel,60–80,Active:1.07 ± 0.53 Sham: 1.20 ± 0.59,15.0,12.0,63.0,63.87 ± 6.85,67.6 ± 7.88,...,ADAS,0.702915,RAVLT,0.590,Bilateral cerebellum,"[30, -88, -35], [-30, -88, -35]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...,Yao_et_al___2022
11,"Hu et al., 2022",10.1016/j.brs.2022.11.009,Parallel,60–90,2,21.0,21.0,76.0,76.86 ± 6.07,75.33 ± 5.73,...,MMSE,0.511000,NaN,NaN,Bilateral angular gyrus,"[63, -53, 27], [-63, -53, 27]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...,Hu_et_al___2022
12,"Padala et al., 2020",10.1016/j.psychres.2017.12.063,Parallel,≧55,NaN,9.0,11.0,74.0,74.3 ± 5.7,79.6 ± 7.7,...,MMSE,0.280000,NaN,NaN,Left DLPFC,"[-22.7, 38.9, 44]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...,Padala_et_al___2020
13,"Cotelli et al., 2011",10.1136/jnnp.2009.197848,Parallel,NaN,NaN,5.0,5.0,71.0,71.2 ± 6.1,74.4 ± 3.8,...,MMSE,-0.080000,NaN,NaN,Left DLPFC,"[-44, 36, 35]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...,Cotelli_et_al___2011



# Preprocess Your Data

**Handle NANs**
- Set drop_nans=True is you would like to remove NaNs from data
- Provide a column name or a list of column names to remove NaNs from

In [4]:
data_df.columns

Index(['Study', 'DOI', 'RCT_Design', 'Age_range', 'CDR', 'Active_N', 'Sham_N',
       'Mean_Age', 'Active_Age', 'Shame_Age', 'Active_M_F', 'Sham_M_F',
       'Active_Education__Yrs_', 'Sham_Education__Yrs_', 'Active_AD_Treatment',
       'Diagnosis', 'Diagnostic_Criteria', 'Stim_Type', 'Frequency',
       'Intensity', 'Navigation', 'Total_Pulses_session', 'session',
       'Total_pulses', 'Overall_Cognition_Test', 'Overall_Cognition_Cohen_s_D',
       'Verbal_Memory_Test', 'Verbal_Memory_Cohen_s_D', 'Stimulation_site',
       'Stim_Coordinate', 'Included', 'Exclusion_Reason', 'roi_path'],
      dtype='object')

In [5]:
coordinate_col = ['Stim_Coordinate']

In [6]:
data_df = cal_palm.drop_nans_from_columns(columns_to_drop_from=coordinate_col)
data_df

,Study,DOI,RCT_Design,Age_range,CDR,Active_N,Sham_N,Mean_Age,Active_Age,Shame_Age,...,Total_pulses,Overall_Cognition_Test,Overall_Cognition_Cohen_s_D,Verbal_Memory_Test,Verbal_Memory_Cohen_s_D,Stimulation_site,Stim_Coordinate,Included,Exclusion_Reason,roi_path
0,"Koch et al., 2022",10.1093/brain/awac285,Parallel,50–85,0.5–1,25.0,25.0,75.0,75 ± 5.6,72.3 ± 7.2,...,51200.0,ADAS,0.359000,NaN,NaN,Precuneus,"[0, -70, 44]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...
1,"Liu et al., 2022",10.1093/cercor/bhab371,Parallel,50–85,0.5–1,25.0,12.0,67.0,67.28 ± 7.74,72.08 ± 7.3,...,28800.0,ADAS,0.839000,NaN,NaN,Bilateral dorsal AG,"[-61, -51, 39], [61, -51, 39]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...
2,"Li et al., 2021",10.1016/j.brs.2021.01.012,Parallel,NaN,NaN,37.0,38.0,65.0,65.97 ± 8.47,64.58 ± 7.88,...,60000.0,ADAS,0.546000,NaN,NaN,Left DLPFC,"[-44,40,29]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...
3,"Wu et al., 2022",10.1016/j.brs.2021.11.007,Parallel,≧50,≦2,24.0,23.0,66.0,66.46 ± 8.25,66.35 ± 7.99,...,25200.0,MMSE,0.608000,RAVLT,0.876,Left DLPFC,"[-38, 44, 26]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...
4,"Zhao et al., 2017",10.18632/oncotarget.13060,Parallel,NaN,1–2,17.0,13.0,69.0,69.3 ± 5.8,71.4 ± 5.2,...,12000.0,ADAS,0.426000,NaN,NaN,Parietal lobe (P3/P4) and posterior temporal l...,"[-39.5, -76.3, 47.4], [59.3, -67.3, 3.8]",0.0,Multifocal,/Users/cu135/Partners HealthCare Dropbox/Calvi...
8,"Wei et al., 2022",10.1016/j.psychres.2022.114721,Parallel,55–85,0.5–2,29.0,27.0,70.0,70 ± 8.68,71.67 ± 7.16,...,8000.0,ADAS,0.100000,PVLT,-0.022,Left lateral parietal location,"[-47, -68, 36]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...
9,"Yao et al., 2022",10.1016/j.brs.2022.06.004,Parallel,60–80,Active:1.07 ± 0.53 Sham: 1.20 ± 0.59,15.0,12.0,63.0,63.87 ± 6.85,67.6 ± 7.88,...,40000.0,ADAS,0.702915,RAVLT,0.590,Bilateral cerebellum,"[30, -88, -35], [-30, -88, -35]",NaN,NaN,NaN
11,"Hu et al., 2022",10.1016/j.brs.2022.11.009,Parallel,60–90,2,21.0,21.0,76.0,76.86 ± 6.07,75.33 ± 5.73,...,48000.0,MMSE,0.511000,NaN,NaN,Bilateral angular gyrus,"[63, -53, 27], [-63, -53, 27]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...
12,"Padala et al., 2020",10.1016/j.psychres.2017.12.063,Parallel,≧55,NaN,9.0,11.0,74.0,74.3 ± 5.7,79.6 ± 7.7,...,60000.0,MMSE,0.280000,NaN,NaN,Left DLPFC,"[-22.7, 38.9, 44]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...
13,"Cotelli et al., 2011",10.1136/jnnp.2009.197848,Parallel,NaN,NaN,5.0,5.0,71.0,71.2 ± 6.1,74.4 ± 3.8,...,20000.0,MMSE,-0.080000,NaN,NaN,Left DLPFC,"[-44, 36, 35]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...


Choose a radius (mm)

In [7]:
radius = 5

In [8]:
data_df.columns

Index(['Study', 'DOI', 'RCT_Design', 'Age_range', 'CDR', 'Active_N', 'Sham_N',
       'Mean_Age', 'Active_Age', 'Shame_Age', 'Active_M_F', 'Sham_M_F',
       'Active_Education__Yrs_', 'Sham_Education__Yrs_', 'Active_AD_Treatment',
       'Diagnosis', 'Diagnostic_Criteria', 'Stim_Type', 'Frequency',
       'Intensity', 'Navigation', 'Total_Pulses_session', 'session',
       'Total_pulses', 'Overall_Cognition_Test', 'Overall_Cognition_Cohen_s_D',
       'Verbal_Memory_Test', 'Verbal_Memory_Cohen_s_D', 'Stimulation_site',
       'Stim_Coordinate', 'Included', 'Exclusion_Reason', 'roi_path'],
      dtype='object')

In [9]:
# import pandas as pd
# data_df = pd.concat([data_df.head(14), data_df.tail(14)], ignore_index=True)

In [10]:
import string

for i, row in data_df.iterrows():
    subname = row['Study'].translate(str.maketrans(string.punctuation, '_' * len(string.punctuation)))
    subname = subname.replace(' ', '_')
    data_df.loc[i, 'Subject'] = subname
data_df

,Study,DOI,RCT_Design,Age_range,CDR,Active_N,Sham_N,Mean_Age,Active_Age,Shame_Age,...,Overall_Cognition_Test,Overall_Cognition_Cohen_s_D,Verbal_Memory_Test,Verbal_Memory_Cohen_s_D,Stimulation_site,Stim_Coordinate,Included,Exclusion_Reason,roi_path,Subject
0,"Koch et al., 2022",10.1093/brain/awac285,Parallel,50–85,0.5–1,25.0,25.0,75.0,75 ± 5.6,72.3 ± 7.2,...,ADAS,0.359000,NaN,NaN,Precuneus,"[0, -70, 44]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...,Koch_et_al___2022
1,"Liu et al., 2022",10.1093/cercor/bhab371,Parallel,50–85,0.5–1,25.0,12.0,67.0,67.28 ± 7.74,72.08 ± 7.3,...,ADAS,0.839000,NaN,NaN,Bilateral dorsal AG,"[-61, -51, 39], [61, -51, 39]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...,Liu_et_al___2022
2,"Li et al., 2021",10.1016/j.brs.2021.01.012,Parallel,NaN,NaN,37.0,38.0,65.0,65.97 ± 8.47,64.58 ± 7.88,...,ADAS,0.546000,NaN,NaN,Left DLPFC,"[-44,40,29]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...,Li_et_al___2021
3,"Wu et al., 2022",10.1016/j.brs.2021.11.007,Parallel,≧50,≦2,24.0,23.0,66.0,66.46 ± 8.25,66.35 ± 7.99,...,MMSE,0.608000,RAVLT,0.876,Left DLPFC,"[-38, 44, 26]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...,Wu_et_al___2022
4,"Zhao et al., 2017",10.18632/oncotarget.13060,Parallel,NaN,1–2,17.0,13.0,69.0,69.3 ± 5.8,71.4 ± 5.2,...,ADAS,0.426000,NaN,NaN,Parietal lobe (P3/P4) and posterior temporal l...,"[-39.5, -76.3, 47.4], [59.3, -67.3, 3.8]",0.0,Multifocal,/Users/cu135/Partners HealthCare Dropbox/Calvi...,Zhao_et_al___2017
8,"Wei et al., 2022",10.1016/j.psychres.2022.114721,Parallel,55–85,0.5–2,29.0,27.0,70.0,70 ± 8.68,71.67 ± 7.16,...,ADAS,0.100000,PVLT,-0.022,Left lateral parietal location,"[-47, -68, 36]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...,Wei_et_al___2022
9,"Yao et al., 2022",10.1016/j.brs.2022.06.004,Parallel,60–80,Active:1.07 ± 0.53 Sham: 1.20 ± 0.59,15.0,12.0,63.0,63.87 ± 6.85,67.6 ± 7.88,...,ADAS,0.702915,RAVLT,0.590,Bilateral cerebellum,"[30, -88, -35], [-30, -88, -35]",NaN,NaN,NaN,Yao_et_al___2022
11,"Hu et al., 2022",10.1016/j.brs.2022.11.009,Parallel,60–90,2,21.0,21.0,76.0,76.86 ± 6.07,75.33 ± 5.73,...,MMSE,0.511000,NaN,NaN,Bilateral angular gyrus,"[63, -53, 27], [-63, -53, 27]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...,Hu_et_al___2022
12,"Padala et al., 2020",10.1016/j.psychres.2017.12.063,Parallel,≧55,NaN,9.0,11.0,74.0,74.3 ± 5.7,79.6 ± 7.7,...,MMSE,0.280000,NaN,NaN,Left DLPFC,"[-22.7, 38.9, 44]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...,Padala_et_al___2020
13,"Cotelli et al., 2011",10.1136/jnnp.2009.197848,Parallel,NaN,NaN,5.0,5.0,71.0,71.2 ± 6.1,74.4 ± 3.8,...,MMSE,-0.080000,NaN,NaN,Left DLPFC,"[-44, 36, 35]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...,Cotelli_et_al___2011


A) Generate Spheres from a column that contains coordinates in lists. Ex) coord col: [x,y,z], [x,y,z]

In [13]:
from calvin_utils.nifti_utils.generate_spherical_roi import SphereROIBuilder
roi_builder = SphereROIBuilder(mask_path=mask_path, out_dir=out_dir, radius=radius)
df = roi_builder.generate_from_coordlist_column(df=data_df, coord_col='Stim_Coordinate', sub_col='Subject', session_col=None, project_on_brain=True, save_individual_roi=False)
df

,Study,DOI,RCT_Design,Age_range,CDR,Active_N,Sham_N,Mean_Age,Active_Age,Shame_Age,...,Overall_Cognition_Test,Overall_Cognition_Cohen_s_D,Verbal_Memory_Test,Verbal_Memory_Cohen_s_D,Stimulation_site,Stim_Coordinate,Included,Exclusion_Reason,roi_path,Subject
0,"Koch et al., 2022",10.1093/brain/awac285,Parallel,50–85,0.5–1,25.0,25.0,75.0,75 ± 5.6,72.3 ± 7.2,...,ADAS,0.359000,NaN,NaN,Precuneus,"[0, -70, 44]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...,Koch_et_al___2022
1,"Liu et al., 2022",10.1093/cercor/bhab371,Parallel,50–85,0.5–1,25.0,12.0,67.0,67.28 ± 7.74,72.08 ± 7.3,...,ADAS,0.839000,NaN,NaN,Bilateral dorsal AG,"[-61, -51, 39], [61, -51, 39]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...,Liu_et_al___2022
2,"Li et al., 2021",10.1016/j.brs.2021.01.012,Parallel,NaN,NaN,37.0,38.0,65.0,65.97 ± 8.47,64.58 ± 7.88,...,ADAS,0.546000,NaN,NaN,Left DLPFC,"[-44,40,29]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...,Li_et_al___2021
3,"Wu et al., 2022",10.1016/j.brs.2021.11.007,Parallel,≧50,≦2,24.0,23.0,66.0,66.46 ± 8.25,66.35 ± 7.99,...,MMSE,0.608000,RAVLT,0.876,Left DLPFC,"[-38, 44, 26]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...,Wu_et_al___2022
4,"Zhao et al., 2017",10.18632/oncotarget.13060,Parallel,NaN,1–2,17.0,13.0,69.0,69.3 ± 5.8,71.4 ± 5.2,...,ADAS,0.426000,NaN,NaN,Parietal lobe (P3/P4) and posterior temporal l...,"[-39.5, -76.3, 47.4], [59.3, -67.3, 3.8]",0.0,Multifocal,/Users/cu135/Partners HealthCare Dropbox/Calvi...,Zhao_et_al___2017
8,"Wei et al., 2022",10.1016/j.psychres.2022.114721,Parallel,55–85,0.5–2,29.0,27.0,70.0,70 ± 8.68,71.67 ± 7.16,...,ADAS,0.100000,PVLT,-0.022,Left lateral parietal location,"[-47, -68, 36]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...,Wei_et_al___2022
9,"Yao et al., 2022",10.1016/j.brs.2022.06.004,Parallel,60–80,Active:1.07 ± 0.53 Sham: 1.20 ± 0.59,15.0,12.0,63.0,63.87 ± 6.85,67.6 ± 7.88,...,ADAS,0.702915,RAVLT,0.590,Bilateral cerebellum,"[30, -88, -35], [-30, -88, -35]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...,Yao_et_al___2022
11,"Hu et al., 2022",10.1016/j.brs.2022.11.009,Parallel,60–90,2,21.0,21.0,76.0,76.86 ± 6.07,75.33 ± 5.73,...,MMSE,0.511000,NaN,NaN,Bilateral angular gyrus,"[63, -53, 27], [-63, -53, 27]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...,Hu_et_al___2022
12,"Padala et al., 2020",10.1016/j.psychres.2017.12.063,Parallel,≧55,NaN,9.0,11.0,74.0,74.3 ± 5.7,79.6 ± 7.7,...,MMSE,0.280000,NaN,NaN,Left DLPFC,"[-22.7, 38.9, 44]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...,Padala_et_al___2020
13,"Cotelli et al., 2011",10.1136/jnnp.2009.197848,Parallel,NaN,NaN,5.0,5.0,71.0,71.2 ± 6.1,74.4 ± 3.8,...,MMSE,-0.080000,NaN,NaN,Left DLPFC,"[-44, 36, 35]",NaN,NaN,/Users/cu135/Partners HealthCare Dropbox/Calvi...,Cotelli_et_al___2011


B) Generate Spheres from several columns that have X, Y, and Z in them. Ex) x_col: x | y_col: y | z_col: z

In [ ]:
out_dir

In [ ]:
radius = 3

In [ ]:
from calvin_utils.nifti_utils.generate_spherical_roi import SphereROIBuilder
roi_builder = SphereROIBuilder(mask_path=None, out_dir=out_dir, radius=radius)
df = roi_builder.generate_from_separate_coord_cols(data_df, x_col='x_mni', y_col='y_mni', z_col='z_mni', sub_col='SUBJECT', session_col=None, project_on_brain=True)
df

Save new DF with file paths

In [15]:
df.to_excel(input_csv_path, index=False)

# Option B - Generate Sphere Manually

In [ ]:
out_dir = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/cognition_2023/revisions/notebook06/ongoing_ungodly_amount_of_edits'
mask_path = '/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii'

Enter coordinates of sphere

In [ ]:
x = -7
y = -46
z = 10
radius=8

Generate the Sphere at The Coordinate

In [ ]:
import nibabel as nib
import numpy as np
brain_img = nib.load(mask_path)
mni_arr = np.array([x,y,z])

In [ ]:
from calvin_utils.nifti_utils.generate_spherical_roi import SphereROIBuilder

SROI = SphereROIBuilder(mask_path=mask_path, out_dir=out_dir, radius=radius)
img = SROI.generate_from_single_coord(centre_xyz=[x,y,z],subject='sphere_boi', session=None, project_on_brain=True, suffix='')
img

Done
- Calvin